In [6]:
# %pip install kagglehub

In [ ]:
import pandas as pd
import kagglehub
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from WiSARD import WiSARD
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
# Download latest version
path = kagglehub.dataset_download("juice0lover/users-vs-bots-classification")
path = f"{path}\\bots_vs_users.csv"
dataset_bot = pd.read_csv(path)

In [68]:
dataset_bot = dataset_bot.dropna()

1 is a bot and 0 is a real user

## 1. Estudo do dataset

In [61]:
dataset_bot_filtered[dataset_bot_filtered['target'] == 1]['avg_comments'].drop_duplicates()

530     0.00
968     0.20
998     0.90
1026    1.30
1246    0.75
1462    0.05
1508    0.10
Name: avg_comments, dtype: float64

In [ ]:
dataset_bot[dataset_bot['target'] == 0].dropna().groupby('has_personal_data').count()

,has_domain,has_birth_date,has_photo,can_post_on_wall,can_send_message,has_website,gender,has_short_name,has_first_name,has_last_name,...,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness,city,has_occupation,occupation_type_university,occupation_type_work
has_personal_data,,,,,,,,,,,,,,,,,,,,,
1.0,521,521,521,521,521,521,521,521,521,521,...,521,521,521,521,521,521,521,521,521,521
Unknown,562,562,562,562,562,562,562,562,562,562,...,562,562,562,562,562,562,562,562,562,562


In [ ]:
bot_columns = dataset_bot[dataset_bot['target'] == 1].dropna().drop_duplicates()
bot_columns.describe()

,target,posts_count,avg_likes,links_ratio,hashtags_ratio,avg_keywords,avg_text_length,attachments_ratio,avg_comments,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness
count,308.0,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000,308.000000
mean,1.0,18.970779,0.256591,0.285487,0.204156,20.504773,1336.640357,0.923084,0.010877,0.861266,0.003604,7.799253,13.568766,0.000325,0.597013
std,0.0,3.605432,1.607582,0.227782,0.151256,16.650971,1059.166339,0.110066,0.100180,0.321154,0.015325,28.342242,56.206353,0.004023,0.210478
min,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,20.000000,0.000000,0.100000,0.100000,5.850000,385.587500,0.900000,0.000000,1.000000,0.000000,0.750000,0.000000,0.000000,0.590000
50%,1.0,20.000000,0.000000,0.200000,0.200000,17.625000,1198.275000,0.950000,0.000000,1.000000,0.000000,1.800000,0.010000,0.000000,0.650000
75%,1.0,20.000000,0.000000,0.500000,0.300000,35.187500,2162.225000,1.000000,0.000000,1.000000,0.000000,4.250000,0.020000,0.000000,0.700000
max,1.0,20.000000,19.450000,1.000000,1.000000,57.950000,4546.900000,1.000000,1.300000,1.000000,0.100000,319.300000,512.280000,0.050000,1.000000


avg_likes baixo
avg_comments baixo
avg_keywords maior, os post tem mais palavras
avg_text_length maior, os textos tem mais palavras
posting_frequency_days menor, posta em menos dias
avg_views baixo, poucas visualizações
reposts_ratio maior, mais repost (não muito tbm )

In [ ]:
users_columns = dataset_bot[dataset_bot['target'] == 0].dropna().drop_duplicates()
users_columns.describe()

,target,posts_count,avg_likes,links_ratio,hashtags_ratio,avg_keywords,avg_text_length,attachments_ratio,avg_comments,reposts_ratio,ads_ratio,avg_views,posting_frequency_days,phone_numbers_ratio,avg_text_uniqueness
count,1083.0,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000,1083.000000
mean,0.0,18.531856,25.040148,0.119769,0.126787,4.097765,319.542798,0.888227,0.781431,0.312022,0.002401,799.032336,81.335125,0.000637,0.688199
std,0.0,4.390551,46.091300,0.178860,0.192734,5.009376,410.798636,0.182912,2.765973,0.327411,0.036655,1795.172334,170.111563,0.010604,0.231164
min,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,20.000000,4.250000,0.000000,0.000000,0.900000,73.685000,0.850000,0.000000,0.000000,0.000000,169.625000,15.320000,0.000000,0.580000
50%,0.0,20.000000,14.750000,0.050000,0.050000,2.550000,195.550000,0.950000,0.000000,0.200000,0.000000,433.800000,46.340000,0.000000,0.750000
75%,0.0,20.000000,32.525000,0.150000,0.150000,5.425000,412.875000,1.000000,0.520000,0.550000,0.000000,951.575000,103.300000,0.000000,0.860000
max,0.0,20.000000,894.900000,1.000000,1.000000,61.600000,5841.250000,1.000000,64.050000,1.000000,0.950000,41415.700000,2866.590000,0.300000,1.000000


In [ ]:
dataset_bot_features = dataset_bot[['avg_comments', 'avg_likes', 'avg_keywords', 'avg_views', 'avg_text_length', 'all_posts_visible',  'is_profile_closed', 'has_hometown',  'all_posts_visible', 'has_career', 'has_personal_data', 'has_interests']] 

## 2. Transformação dos dados 

In [86]:
dataset_bot_features = dataset_bot[['avg_comments', 'avg_likes', 'avg_keywords', 'avg_views', 'avg_text_length',  'is_profile_closed', 'has_hometown',  'all_posts_visible', 'has_career', 'has_personal_data', 'has_interests','target']].reset_index()

has_career - 80:90

### 2.1 retirada de valores não numéricos 

In [87]:
dataset_bot_features.loc[:, 'has_personal_data'] = dataset_bot_features['has_personal_data'].replace('Unknown', 0.0)
dataset_bot_features.loc[:, 'has_interests'] = dataset_bot_features['has_interests'].replace('Unknown', 2.0)
dataset_bot_features.loc[:, 'has_career'] = dataset_bot_features['has_career'].replace('Unknown', 2.0)
dataset_bot_features.loc[:, 'all_posts_visible'] = dataset_bot_features['all_posts_visible'].replace('Unknown', 2.0)
dataset_bot_features.loc[:, 'has_hometown'] = dataset_bot_features['has_hometown'].replace('Unknown', 2.0)
dataset_bot_features.loc[:, 'is_profile_closed'] = dataset_bot_features['is_profile_closed'].replace('Unknown', 1.0)
dataset_bot_features = dataset_bot_features.astype(float).drop(columns = 'index')


In [ ]:
import numpy as np


# Filtrando o dataset para incluir apenas as colunas de interesse
dataset_bot_filtered =dataset_bot_features

columns_of_interest = dataset_bot_features.columns
# Plotando a distribuição dos valores para cada coluna com base no target (0 ou 1)
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(columns_of_interest[:-1]):  # Excluindo a coluna target
    ax = axes[i]
    data_1 = dataset_bot_filtered[dataset_bot_filtered['target'] == 1][col]
    data_0 = dataset_bot_filtered[dataset_bot_filtered['target'] == 0][col]
    
    bins = np.histogram_bin_edges(np.concatenate([data_1, data_0]), bins=30)
    counts_1, _ = np.histogram(data_1, bins=bins)
    counts_0, _ = np.histogram(data_0, bins=bins)
    width = (bins[1] - bins[0]) * 0.4  # largura das barras
    
    ax.bar(bins[:-1] - width/2, counts_1, width=width, alpha=0.7, label='Bot', color='blue', align='edge')
    ax.bar(bins[:-1] + width/2, counts_0, width=width, alpha=0.7, label='Usuários', color='red', align='edge')
    ax.set_title(f'Distribuição de {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequência')
    ax.legend()

plt.tight_layout()
plt.show()


### 2.3 Separação de sets

In [88]:
#Separação dos dados de treino e teste usando sklearn
from sklearn.model_selection import train_test_split

X = dataset_bot_features.drop('target', axis=1)
y = dataset_bot_features['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.reset_index()[['target']]
y_test = y_test.reset_index()[['target']]

### 2.4 Binarização

In [89]:
import numpy as np

def thermometer_encoding(df, n_bits=10):
    binary_matrix = []
    
    for col in df.columns:
        col_vals = df[col].values
        
        # Normalizando os valores para o intervalo [0, 1]
        min_val = np.min(col_vals)
        max_val = np.max(col_vals)
        range_val = max_val - min_val
        
        # Caso o range seja zero (coluna constante), tratar separadamente
        if range_val == 0:
            col_encoded = np.zeros((len(col_vals), n_bits))  # Se todos os valores são iguais, codificar como 0
        else:
            # Aplicando a codificação termômetro com base na normalização
            col_encoded = np.array([
                [1 if i < int(((val - min_val) / range_val) * n_bits) else 0 for i in range(n_bits)]
                for val in col_vals
            ])
        
        # Adicionando a matriz binária codificada para a coluna
        binary_matrix.append(col_encoded)
    
    # Combinando todas as colunas binarizadas horizontalmente
    return np.hstack(binary_matrix)


In [90]:
X_train_bin = pd.DataFrame( thermometer_encoding(X_train, n_bits=10))
X_test_bin = pd.DataFrame( thermometer_encoding(X_test, n_bits=10))

avg_keywords é de 20:30

## 3. Treinamento sem Regra

In [20]:
input_size = X_train_bin.shape[1]
topologia_do_input = ['x' + str(i) for i in range(input_size)]

In [21]:
wisard = WiSARD(topologia_do_input, tuple_size= 10) 

# wisard.train(X_train_bin, y_train)

In [22]:
X_train_bin_fl = [np.array(row) for row in X_train_bin.values]
X_test_bin_fl = [np.array(row) for row in X_test_bin.values]
y_train_fl = y_train['target'].values
y_test_fl = y_test['target'].values

In [23]:
wisard.train(X_train_bin_fl, y_train_fl)

In [24]:
pred = wisard.predict(X_test_bin_fl, bleaching_start=0)

In [25]:
from sklearn.metrics import accuracy_score
import numpy as np

# Converter para array se necessário
y_test = np.array(y_test)
y_pred = np.array(pred)

# Substituir "não identificado" por -1.0
y_pred_float = np.where(y_pred == "não identificado", -1.0, y_pred).astype(float)
y_test_float = y_test.astype(float)

# Calcular acurácia
acuracia = accuracy_score(y_test_float, y_pred_float)

print(f"Acurácia (incluindo 'não identificado' como classe -1): {acuracia:.4f}")


Acurácia (incluindo 'não identificado' como classe -1): 0.5125


## 4. Treinamento com Regra

In [96]:
input_size = X_train_bin.shape[1]
topologia_do_input = [['x' + str(i)] for i in range(input_size)]

In [97]:
regras_bot = {
0: [
     ( #avg_keywoard é menor que 20 
        ['x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29'],
        lambda x20, x21, x22, x23, x24, x25, x26, x27, x28, x29:
            x20 and not x21 and not x22 and not x23 and not x24 and not x25 and not x26 and not x27 and not x28 and not x29 or
            x20 and x21 and not x22 and not x23 and not x24 and not x25 and not x26 and not x27 and not x28 and not x29
    ),
    ( # has_carer 1 ou 2
    ['x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89'],
    lambda x80, x81, x82, x83, x84, x85, x86, x87, x88, x89:
        x80 and x81 and  x82 and  x83 and  x84 and not x85 and not x86 and not x87 and not x88 and not x89 or
        x80 and x81 and x82 and  x83 and x84 and x85 and  x86 and x87 and  x88 and x89
)

],
1: [
     ( #avg_keywoard é maior que 30
        ['x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29'],
        lambda x20, x21, x22, x23, x24, x25, x26, x27, x28, x29:
            x20 and  x21 and  x22 and  x23 and  x24 and  x25 and  x26 and  x27 and  x28 and  x29 or
            x20 and  x21 and  x22 and  x23 and  x24 and  x25 and  x26 and  x27 and  x28 and not  x29

    )
]
}

Regra 1: Se avg_comments, avg_views e avg_likes são baixos e  avg_keywords e avg_text_length são altos, então é um Bot

Regra 2: Se o Perfil é aberto,  todos os post são públicos, has_interests vazio, não tem carreira, não tem endereço e has_personal_data vazio, então é um bot

In [106]:
wisard = WiSARD(topologia_do_input, tuple_size= 6,  threshold = 0, regra = regras_bot, alfa = 900) 

X_train_bin_fl = [np.array(row) for row in X_train_bin.values]
X_test_bin_fl = [np.array(row) for row in X_test_bin.values]
y_train_fl = y_train['target'].values
y_test_fl = y_test['target'].values

wisard.train(X_train_bin_fl, y_train_fl)
pred = wisard.predict(X_test_bin_fl, bleaching_start=1)

In [107]:
from sklearn.metrics import accuracy_score
import numpy as np

# Converter para array se necessário
y_val = np.array(y_test)
y_pred = np.array(pred)

# Substituir "não identificado" por -1.0
y_pred_float = np.where(y_pred == "não identificado", -1.0, y_pred).astype(float)
y_test_float = y_val.astype(float)

# Calcular acurácia
acuracia = accuracy_score(y_test_float, y_pred_float)

print(f"Acurácia (incluindo 'não identificado' como classe -1): {acuracia:.4f}")

Acurácia (incluindo 'não identificado' como classe -1): 0.8136
